In [228]:
import numpy as np
import pandas as pd
from scipy.stats.stats import pearsonr # used to calculate correlation coefficient
from pymongo import MongoClient
import json
from datetime import datetime, timedelta, date
from pprint import pprint
from enum import Enum
import copy
from collections import deque
from time import time

# Read timeseries of benchmark and coin

In [119]:
filename_benchmark = 'testing/BTC_Bitfinex_USD.csv'
filename_coin      = 'testing/XRP_Bitfinex_USD.csv'

# --- read benchmark ---
df_benchmark = pd.read_csv(filename_benchmark, header=None, sep=";", 
                            names=['date', 'timestamp', 'open', 'high', 'low', 'close', 'volFrom', 'volTo'])
    
# this makes indexing via date faster
df_benchmark = df_benchmark.set_index(['date'])         # index: string
df_benchmark.index = pd.to_datetime(df_benchmark.index) # index: datetime

pprint(df_benchmark)

# --- read coin ---
df_coin = pd.read_csv(filename_coin, header=None, sep=";", 
                      names=['date', 'timestamp', 'open', 'high', 'low', 'close', 'volFrom', 'volTo'])
    
# this makes indexing via date faster
df_coin = df_coin.set_index(['date'])         # index: string
df_coin.index = pd.to_datetime(df_coin.index) # index: datetime

                        timestamp      open      high       low     close  \
date                                                                        
2018-01-01 00:00:00  1.514765e+09   9830.51   9830.51   9830.51   9830.51   
2018-01-01 01:00:00  1.514768e+09   9830.42   9830.42   9830.42   9830.42   
2018-01-01 02:00:00  1.514772e+09   9874.93   9874.93   9874.93   9874.93   
2018-01-01 03:00:00  1.514776e+09   9894.21   9894.21   9894.21   9894.21   
2018-01-01 04:00:00  1.514779e+09   9900.78   9900.78   9900.78   9900.78   
2018-01-01 05:00:00  1.514783e+09  10032.37  10032.37  10032.37  10032.37   
2018-01-01 06:00:00  1.514786e+09  10120.35  10120.35  10120.35  10120.35   
2018-01-01 07:00:00  1.514790e+09  10120.25  10120.25  10120.25  10120.25   
2018-01-01 08:00:00  1.514794e+09  10070.72  10070.72  10070.72  10070.72   
2018-01-01 09:00:00  1.514797e+09  10091.04  10091.04  10091.04  10091.04   
2018-01-01 10:00:00  1.514801e+09  10170.63  10170.63  10170.63  10170.63   

# Calculate Multiplier and Correlation

If there is no pre-defined libraries to compute (Pearson) correlation, it can also be calculated by its definition: <br>
https://en.wikipedia.org/wiki/Pearson_correlation_coefficient

This relies on calculating the covariance between two vectors, and the variance of each vector indidually. <br>
If you need further clarification please ask.

In [4]:
#

# input variables

#input parameter
dt_benchmark_startTime    = datetime.datetime.strptime("2018-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")

#always current time
dt_benchmark_endTime      = datetime.datetime.strptime("2018-01-31 23:00:00", "%Y-%m-%d %H:%M:%S")

#input parameter
ReturnFrequency = "daily"

#list of currencies

#END input variables
    
dt_currentTime = dt_benchmark_startTime

# add first interval
dt_previousTime = dt_currentTime
if ReturnFrequency == "hourly":
    dt_currentTime += datetime.timedelta(hours=1)
elif ReturnFrequency == "daily":
    dt_currentTime += datetime.timedelta(days=1)
else:
    print('ERROR. Need to implment other frequencies')
    assert(False)


arr_PnL_benchmark  = np.array([])
arr_PnL_coin       = np.array([])
   
#-----------------------------------------#
#          calculate return timeseries    #   
#-----------------------------------------#

while (dt_currentTime <= dt_benchmark_endTime):
    # calculate return of benchmark in period [t-1, t]
    PnL_benchmark = df_benchmark.loc[dt_currentTime]['close'] / \
                    df_benchmark.loc[dt_previousTime]['close'] -1.0
    arr_PnL_benchmark = np.append(arr_PnL_benchmark, PnL_benchmark)
    
    # calculate return of strategy in period [t-1, t] (based on equity, i.e. MtM value of positions)
    PnL_coin = df_coin.loc[dt_currentTime]['close']  / \
               df_coin.loc[dt_previousTime]['close'] -1.0
    arr_PnL_coin = np.append(arr_PnL_coin, PnL_coin)
     
    # move to next timepoint
    if ReturnFrequency == "hourly":
        dt_previousTime += datetime.timedelta(hours=1)
        dt_currentTime  += datetime.timedelta(hours=1)
    elif ReturnFrequency == "daily":
        dt_previousTime += datetime.timedelta(days=1)
        dt_currentTime += datetime.timedelta(days=1)
    else:
        print('ERROR. Need to implment other frequencies')
        assert(False)

#-----------------------------------------#
#          calculate multiplier           #   
#-----------------------------------------#
arr_x = arr_PnL_benchmark
arr_y = arr_PnL_coin

# least square regression (linear): y = alpha + beta*x
linReg = np.polyfit(x=arr_PnL_benchmark, y=arr_PnL_coin, deg=1)

alpha = linReg[1] # this is the y-intercept, not needed
beta  = linReg[0] # this is the slope, which also is the multiplier
multiplier = beta
print("multiplier            : ", multiplier)

#-----------------------------------------#
#          calculate correlation          #   
#-----------------------------------------#
correlation = pearsonr(arr_PnL_benchmark, arr_PnL_coin)
print("correlation            :", correlation[0])

multiplier            :  -0.6945408340863843
correlation            : -0.15598094157469428


# Multiplier and Correlation class calculator

Class calculates multiplier and correlation matrix


In [488]:
from flask import Flask
from flask import request
import numpy as np
import pandas as pd
from scipy.stats.stats import pearsonr # used to calculate correlation coefficient
from pymongo import MongoClient
import json
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from calendar import monthrange
from pprint import pprint
from enum import Enum
import copy
from collections import deque
import time

class MultiplierCorellationCalculator:
    class RequestFrequency(Enum):
        DAILY  = 0
        HOURLY = 1
    
    class HourlyTimeIntervals(Enum):
        A_DAY       = 1
        FIVE_DAYS   = 5
        WEEK        = 7
        TEN_DAYS    = 10
        TWO_WEEKS   = 14
    
    class DailyTimeIntervals(Enum):
        A_MONTH      = 1
        THREE_MONTHS = 3
        HALF_YEAR    = 6
        NINE_MONTHS  = 9
        A_YEAR       = 12
    
    FREQUENCY_LIST        = RequestFrequency.__members__.keys()
    HOURLY_TIME_INTERVALS = list(map(lambda x: x.value, HourlyTimeIntervals.__members__.values()))
    DAILY_TIME_INTERVALS  = list(map(lambda x: x.value, DailyTimeIntervals.__members__.values()))
    TIME_INTERVALS_DICT   = {
        'hourly': HOURLY_TIME_INTERVALS, 
        'daily': DAILY_TIME_INTERVALS,
    }
    
    def __init__(self,
                 db_name='darqube_db',
                 time_interval=1,
                 currencies_list='all',
                 return_frequency='daily'):
        if return_frequency.upper() not in self.FREQUENCY_LIST:
            raise Exception('Only [daily, hourly] values supports for return_frequency parameter yet...')
        self.time_points =  self.TIME_INTERVALS_DICT[return_frequency]
        if time_interval not in self.time_points:
            msg = 'Only %s values supports for %s collection' % (','.join(self.time_points), 
                                                                 return_frequency)
            raise Exception(msg)
        self.return_frequency = "%s_data" % return_frequency    # select correct collection name 
        self.mongo_c = None
        self._mongo_connect(db_name)
        self.db = self.mongo_c[db_name]
        self.currencies_list  = currencies_list
        self.collection       = self.db[self.return_frequency]
        if currencies_list == 'all':
            self.currencies_list = [x['Ccy'] for x in self.collection.find({},{'Ccy': 1, '_id': 0})]
        self._calculate_time_bounds()
        self.currencies       = {}
    
    
    def _preprocess_time_intervals(self): 
        if return_frequency == 'daily':
            return list(map(lambda x: x * 30, self.time_points))
        else:
            return list(map(lambda x: x * 24, self.time_points))
    
    
    def recalculate_pairs(self):
        """
        Example for daily:
            
        'm_and_c_matrix': {
            '1': [  {   ccy: 'ETH',
                        multiplier: 0.5,
                        correlation: 0.93 },
                    {   ccy: 'LTC',
                        mult: 0.5,
                        corr: 0.93 }
            ],
            '5': [  {   ccy: 'ETH',
                        multiplier: 0.5,
                        correlation: 0.93 },
                    {   ccy: 'LTC',
                        mult: 0.5,
                        corr: 0.93 }
            ],
            ......
            '14': [  {   ccy: 'ETH',
                        multiplier: 0.5,
                        correlation: 0.93 },
                    {   ccy: 'LTC',
                        mult: 0.5,
                        corr: 0.93 }
            ]
        }
        """
        currencies_list  = deque(self.currencies_list)
        time_intervals   = self._preprocess_time_intervals()
        pairs_multiplier_correlation = {}
        for benchmark_currency in currencies_list:
            matrix = {}
            for n_times, point in zip(time_intervals, self.time_points):
                pairs = []
                for coin_currency in [x for x in currencies_list if x != benchmark_currency]:
                    multiplier, correlation = self.calculate_for_pair(
                                                benchmark_currency, 
                                                coin_currency, 
                                                n_times)
                    pair = { 'ccy': coin_currency,
                             'multiplier': multiplier, 
                             'correlation': correlation }
                    pairs.append(pair)
                matrix[str(point)] = pairs
                
            self.db[self.return_frequency].update(
               { 'Ccy': benchmark_currency },
               { 
                 '$set': { "m_and_c_matrix": matrix}
               },
                upsert=True
            )

            
    def _calculate_time_bounds(self):
        pipeline = [
            {
                '$unwind': "$history"
            },
            { 
                '$group' : 
                { 
                    '_id': "$Ccy",
                    'max': { '$max' : "$history.time" },
                    'min': { '$min' : "$history.time" }
                }
            }
        ]
        self.currencies_min_max_time = { x['_id']: {'min':x['min'], 'max':x['max']} for x in self.collection.aggregate(pipeline)}
        
    
    def _fix_horizon(self, benchmark_ccy, coin_ccy, horizon):
        to_datetime = lambda x: datetime.fromtimestamp(x)
#         print("Bench %s Coin %s" % (benchmark_ccy, coin_ccy))
#         print(self.currencies_min_max_time)
        bc_min = max(self.currencies_min_max_time[benchmark_ccy]['min'],
                     self.currencies_min_max_time[coin_ccy]['min'])
        bc_max = min(self.currencies_min_max_time[benchmark_ccy]['max'],
                     self.currencies_min_max_time[coin_ccy]['max'])
        minln = to_datetime(bc_min)
        maxln = to_datetime(bc_max)
        delta = maxln - minln
        n_times = horizon
        if self.return_frequency == 'daily_data':
            delta = delta.days
        else:
            delta = delta.seconds / 360
        
        if delta > 1 and delta < n_times:
            n_times = delta
        if horizon != n_times:
            print("Horizon for %s/%s pair fixed from %s to %s" % (benchmark_ccy, coin_ccy, horizon, n_times))
        return n_times

    
    def calculate_for_pair(self, benchmark_ccy, coin_ccy, horizon):
        # --- read coin ---
        fixed_horizon                   = self._fix_horizon(benchmark_ccy, 
                                                            coin_ccy,
                                                            horizon)
        arr_PnL_benchmark, arr_PnL_coin = self._timeseries(benchmark_ccy, 
                                                           coin_ccy, 
                                                           fixed_horizon)
        multiplier, correlation         = self._multiplier_and_correlation(arr_PnL_benchmark, 
                                                                           arr_PnL_coin)
        return (multiplier, correlation)
    
    def _multiplier_and_correlation(self, arr_PnL_benchmark, arr_PnL_coin):
        #          calculate multiplier
        # least square regression (linear): y = alpha + beta*x
        linReg = np.polyfit(x=arr_PnL_benchmark, y=arr_PnL_coin, deg=1)
        alpha = linReg[1] # this is the y-intercept, not needed
        beta  = linReg[0] # this is the slope, which also is the multiplier
        multiplier = beta
        print("multiplier            : ", multiplier)
        #          calculate correlation          #   
        correlation = pearsonr(arr_PnL_benchmark, arr_PnL_coin)
        print("correlation            :", correlation[0])
        return (multiplier, correlation[0])
    
    #-----------------------------------------#
    #          calculate return timeseries    # 
    #-----------------------------------------#
    def _timeseries(self, benchmark_ccy, coin_ccy, horizon):
        df_benchmark = self._retrieve_currency_history(benchmark_ccy)
        df_benchmark = df_benchmark.pct_change()
        print(df_benchmark['close'])
        df_benchmark = df_benchmark['close'].values[-horizon:-1]
        print(df_benchmark)
        
        df_coin      = self._retrieve_currency_history(coin_ccy).pct_change()
        df_coin      = df_coin['close'].values[-horizon:-1]
        
        return (df_benchmark, df_coin)


    # --- connect and preprocess utilities for mongo collection --- 
    def _reconstruct_currency_date(self, cur):
        frmt = "{:%Y-%m-%d}"
        if self.return_frequency == 'hourly':
            frmt = "{:%Y-%m-%d %H:%M:%S}"
        for cur_value, index in zip(cur['history'], range(len(cur['history']))):
            #  cur['history'][index]['date'] = datetime.fromtimestamp(cur_value['time'])
            cur['history'][index]['date'] = frmt.format(datetime.fromtimestamp(cur_value['time']))
        return cur

    
    def _mongo_connect(self, db_name):
        if self.mongo_c == None: 
            self.mongo_c = MongoClient('localhost',
                    authSource=db_name)
        return self.mongo_c
        
        
    def _preprocess_collection(self, collection_name, filter_params):
        collection = self.db[collection_name]
        if not collection:
            raise Exception('collection not found')
        return self._reconstruct_currency_date(collection.find_one(filter_params))

    
    def _retrieve_currency_history(self, currency):
        if currency not in self.currencies:
            collection_schema = self.return_frequency # return frequency points to the name of collection
            df_data = self._preprocess_collection(collection_schema, {'Ccy': currency})
            df_data = pd.DataFrame(df_data['history'])
            # this makes indexing via date faster
            df_data = df_data.set_index(['date'])         # index: string
            df_data.index = pd.to_datetime(df_data.index)
            self.currencies[currency] = df_data
        return self.currencies[currency]

    


In [489]:
time_interval = 1

currencies_list = ['BTC', 'ETH', 'BCH', 'XRP', 'EOS']
return_frequency = 'daily'

new_compare = MultiplierCorellationCalculator(
    time_interval=time_interval,
    currencies_list=currencies_list,
    return_frequency=return_frequency)
new_compare.recalculate_pairs()



Bench BTC Coin ETH
{'XRP': {'min': 1421798400, 'max': 1539561600}, 'ETH': {'min': 1438905600, 'max': 1539561600}, 'EOS': {'min': 1498694400, 'max': 1539561600}, 'BTC': {'min': 1279324800, 'max': 1539561600}, 'BCH': {'min': 1501545600, 'max': 1539561600}}
date
2010-07-17         NaN
2010-07-18    0.733791
2010-07-19   -0.058714
2010-07-20   -0.075000
2010-07-21    0.059807
2010-07-22   -0.362454
2010-07-23    0.240000
2010-07-24   -0.129032
2010-07-25   -0.074074
2010-07-26    0.108911
2010-07-27    0.071429
2010-07-28   -0.018333
2010-07-29    0.186757
2010-07-30   -0.103004
2010-07-31    0.082137
2010-08-01   -0.099484
2010-08-02   -0.018003
2010-08-03    0.000000
2010-08-04   -0.050000
2010-08-05    0.070175
2010-08-06    0.021311
2010-08-07   -0.052970
2010-08-08    0.032203
2010-08-09    0.165846
2010-08-10   -0.014085
2010-08-11   -0.042857
2010-08-12    0.044776
2010-08-13   -0.078571
2010-08-14    0.038760
2010-08-15   -0.025522
                ...   
2018-09-16   -0.003011
2018

/Users/rvarnava/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:127: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


multiplier            :  0.8510874885213232
correlation            : 0.7760821570611429
Bench ETH Coin XRP
{'XRP': {'min': 1421798400, 'max': 1539561600}, 'ETH': {'min': 1438905600, 'max': 1539561600}, 'EOS': {'min': 1498694400, 'max': 1539561600}, 'BTC': {'min': 1279324800, 'max': 1539561600}, 'BCH': {'min': 1501545600, 'max': 1539561600}}
date
2015-08-07         NaN
2015-08-08   -0.600000
2015-08-09    0.000000
2015-08-10    0.000000
2015-08-11   -0.175000
2015-08-12    0.303030
2015-08-13    0.457364
2015-08-14   -0.047872
2015-08-15    0.000000
2015-08-16   -0.234637
2015-08-17   -0.051095
2015-08-18    0.046154
2015-08-19   -0.088235
2015-08-20    0.225806
2015-08-21   -0.052632
2015-08-22   -0.027778
2015-08-23   -0.035714
2015-08-24   -0.081481
2015-08-25    0.024194
2015-08-26   -0.070866
2015-08-27   -0.033898
2015-08-28    0.140351
2015-08-29   -0.092308
2015-08-30    0.118644
2015-08-31   -0.007576
2015-09-01    0.038168
2015-09-02   -0.161765
2015-09-03    0.078947
2015-09-

multiplier            :  1.0329635780065747
correlation            : 0.6292646825096632
Bench BCH Coin BTC
{'XRP': {'min': 1421798400, 'max': 1539561600}, 'ETH': {'min': 1438905600, 'max': 1539561600}, 'EOS': {'min': 1498694400, 'max': 1539561600}, 'BTC': {'min': 1279324800, 'max': 1539561600}, 'BCH': {'min': 1501545600, 'max': 1539561600}}
date
2017-08-01         NaN
2017-08-02    0.255697
2017-08-03   -0.264091
2017-08-04   -0.376404
2017-08-05   -0.129696
2017-08-06    0.097912
2017-08-07    0.333774
2017-08-08    0.064569
2017-08-09   -0.123053
2017-08-10   -0.088621
2017-08-11    0.188648
2017-08-12   -0.031447
2017-08-13   -0.065981
2017-08-14    0.016534
2017-08-15   -0.008899
2017-08-16    0.007802
2017-08-17    0.485551
2017-08-18    0.549518
2017-08-19    0.070612
2017-08-20   -0.026092
2017-08-21   -0.158564
2017-08-22    0.131159
2017-08-23   -0.015367
2017-08-24   -0.080480
2017-08-25    0.020409
2017-08-26   -0.017471
2017-08-27   -0.008271
2017-08-28   -0.040857
2017-08-

multiplier            :  0.235763264061228
correlation            : 0.42000747037407604
Bench BCH Coin ETH
{'XRP': {'min': 1421798400, 'max': 1539561600}, 'ETH': {'min': 1438905600, 'max': 1539561600}, 'EOS': {'min': 1498694400, 'max': 1539561600}, 'BTC': {'min': 1279324800, 'max': 1539561600}, 'BCH': {'min': 1501545600, 'max': 1539561600}}
date
2017-08-01         NaN
2017-08-02    0.255697
2017-08-03   -0.264091
2017-08-04   -0.376404
2017-08-05   -0.129696
2017-08-06    0.097912
2017-08-07    0.333774
2017-08-08    0.064569
2017-08-09   -0.123053
2017-08-10   -0.088621
2017-08-11    0.188648
2017-08-12   -0.031447
2017-08-13   -0.065981
2017-08-14    0.016534
2017-08-15   -0.008899
2017-08-16    0.007802
2017-08-17    0.485551
2017-08-18    0.549518
2017-08-19    0.070612
2017-08-20   -0.026092
2017-08-21   -0.158564
2017-08-22    0.131159
2017-08-23   -0.015367
2017-08-24   -0.080480
2017-08-25    0.020409
2017-08-26   -0.017471
2017-08-27   -0.008271
2017-08-28   -0.040857
2017-08-

date
2015-01-21         NaN
2015-01-22    0.051871
2015-01-23   -0.001248
2015-01-24    0.086875
2015-01-25   -0.022427
2015-01-26   -0.135882
2015-01-27    0.055140
2015-01-28   -0.032258
2015-01-29   -0.100000
2015-01-30    0.111111
2015-01-31   -0.000667
2015-02-01   -0.098065
2015-02-02   -0.018491
2015-02-03    0.115298
2015-02-04   -0.054054
2015-02-05    0.123571
2015-02-06   -0.014622
2015-02-07   -0.052903
2015-02-08   -0.027248
2015-02-09   -0.054622
2015-02-10    0.084444
2015-02-11   -0.001366
2015-02-12   -0.021888
2015-02-13    0.010490
2015-02-14   -0.169550
2015-02-15    0.000000
2015-02-16    0.256667
2015-02-17   -0.137268
2015-02-18    0.152191
2015-02-19   -0.128085
                ...   
2018-09-16    0.001069
2018-09-17   -0.037367
2018-09-18    0.179298
2018-09-19    0.023511
2018-09-20    0.373660
2018-09-21    0.258417
2018-09-22    0.015592
2018-09-23   -0.003315
2018-09-24   -0.138631
2018-09-25    0.067060
2018-09-26   -0.023234
2018-09-27    0.059466
2018-0

multiplier            :  0.6248310381292016
correlation            : 0.6885487726134938
Bench EOS Coin BTC
{'XRP': {'min': 1421798400, 'max': 1539561600}, 'ETH': {'min': 1438905600, 'max': 1539561600}, 'EOS': {'min': 1498694400, 'max': 1539561600}, 'BTC': {'min': 1279324800, 'max': 1539561600}, 'BCH': {'min': 1501545600, 'max': 1539561600}}
date
2017-06-29         NaN
2017-06-30    0.072941
2017-07-01    0.151316
2017-07-02    1.857143
2017-07-03    0.400000
2017-07-04   -0.228571
2017-07-05   -0.117284
2017-07-06    0.129371
2017-07-07   -0.179567
2017-07-08   -0.105660
2017-07-09   -0.063291
2017-07-10   -0.234234
2017-07-11   -0.100000
2017-07-12    0.281046
2017-07-13   -0.127551
2017-07-14   -0.116959
2017-07-15   -0.145695
2017-07-16   -0.054264
2017-07-17    0.237705
2017-07-18    0.105960
2017-07-19   -0.131737
2017-07-20    0.179310
2017-07-21    0.046784
2017-07-22    0.134078
2017-07-23   -0.059113
2017-07-24   -0.005236
2017-07-25   -0.121053
2017-07-26   -0.005988
2017-07-

In [368]:
class MultiplierCorrelationRetriever:
    def __init__(self,
                 horizon,
                 currencies_list,
                 return_frequency='daily'):
        self.mongo_c          = None
        self.db_name          = 'darqube_db'
        self._mongo_connect()
        self.return_frequency = "%s_data" % return_frequency
        self.db               = self.mongo_c[self.db_name]
        self._mongo_connect()
        self.horizon          = horizon
        self.currencies_list  = currencies_list
        
        
    def retrieve_data(self):
        currencies_list  = deque(self.currencies_list)
        pairs_multiplier_correlation = {}
        while len(currencies_list) > 1:
            benchmark_currency  = currencies_list.popleft()
            pair = self._retrieve_multiplier_correlation(benchmark_currency, currencies_list)
            pairs_multiplier_correlation = {**pairs_multiplier_correlation, **pair}
        return pairs_multiplier_correlation
        
    def _mongo_connect(self):
        if self.mongo_c == None: 
            self.mongo_c = MongoClient('localhost',
                    authSource=self.db_name)
        return self.mongo_c
        
        
    def _retrieve_collection(self, filter_params):
        collection = self.db[self.return_frequency]
        if not collection:
            raise Exception('collection not found')
        return collection.find_one(filter_params)

    
    def _retrieve_multiplier_correlation(self, benchmark, coins):
        df_data = self._retrieve_collection({'Ccy': benchmark})
        df_data = df_data['m_and_c_matrix'][str(self.horizon)]
        df_data = [x for x in df_data if x['ccy'] in coins]
        df_data = {"%s/%s" %(benchmark, obj['ccy']): {'multiplier': obj['multiplier'], 'correlation': obj['correlation']} for obj in df_data}
        return df_data
        

In [370]:
coins = ['BTC', 'ETH', 'BCH']
new_retriever = MultiplierCorrelationRetriever(horizon=1, 
                                               currencies_list=coins,
                                              return_frequency='daily')
print(new_retriever.retrieve_data())


{'BTC/ETH': {'multiplier': -0.5849820982170724, 'correlation': -0.49828339040150016}, 'BTC/BCH': {'multiplier': 0.231676017041614, 'correlation': 0.19365146361434357}, 'ETH/BCH': {'multiplier': 0.013750641473341176, 'correlation': 0.013493630987402335}}


In [483]:
db = MongoClient()['darqube_db']
collection_data = db['daily_data']
currencies_list = collection_data.distinct('Ccy')
minln = 0
maxln = time.time()
# for curr in currencies_list:
pipeline = [
            {
                '$unwind': "$history"
            },
            { 
                '$group' : 
                { 
                    '_id': "$Ccy",
                    'max': { '$max' : "$history.time" },
                    'min': { '$min' : "$history.time" }
                }
            }
        ]
# max_list = collection_data.aggregate(pipeline)
# for max_list in collection_data.aggregate(pipeline):
#     print(max_list)
currencies_min_max_time = { x['_id']: {'max': x['max'] , 'min': x['min']} for x in collection_data.aggregate(pipeline)}
print(currencies_min_max_time)
        

{'XRP': {'max': 1539561600, 'min': 1421798400}, 'ETH': {'max': 1539561600, 'min': 1438905600}, 'EOS': {'max': 1539561600, 'min': 1498694400}, 'BTC': {'max': 1539561600, 'min': 1279324800}, 'BCH': {'max': 1539561600, 'min': 1501545600}}


In [ ]:

collection_data = self.db[self.return_frequency]
minln = 0
maxln = time.time()
all_curr_max = collection_data.aggregate(
    [{ '$group': 
         { '_id': 0, 
           'max': {'$max' : 
                       {'$max': 
                        {"$history.time"}
                       }
                  }
         }
     }]
    )
all_curr_max = next(all_curr_max)
all_curr_min = collection_data.aggregate([{ '$group' : { '_id': 0, 'time': { '$max' : "$history.time"}}}])
all_curr_min = next(all_curr_min)

maxln = min(max(all_curr_max['time']), maxln)
minln = max(min(t['time']), minln)
return (minln, maxln)

In [441]:
dt1 = datetime.now()
time.sleep(3)
dt2 = datetime.now()
delta = dt2 - dt1
print(delta.seconds)

3
